In [ ]:
from crewai import LLM

llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.7,GEMINI_API_KEY = "")

In [4]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
import json 

@tool
def search_web_tool(query: str):
    """
    Searches the web and returns results.
    """
    search_tool = DuckDuckGoSearchResults(num_results=10, verbose=True)
    return search_tool.run(query)

In [5]:
from crewai import Agent

# Agent Resercher
guide_expert= Agent( 
    role="City Local Guide Expert",
    goal="Provides information on things to do in the city based on the user's interests.",
    backstory="""A local expert with a passion for sharing the best experiences and hidden gems of their city.""",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=llm,
    allow_delegation=False,
    )

location_expert = Agent(
    role="Travel Trip Expert",
    goal="Adapt to the user destination vity language (French if city in French Country. Gather helpful information about to the city and city during travel.",
    backstory="""A seasoned traveler who has explored various destinations and knows the ins and outs of travel logistics.""",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=llm,
    allow_delegation=False,
    )

planner_expert = Agent(
    role="Travel Planning Expert",
    goal="Compiles all gathered information to provide a comprehensive travel plan.",
    backstory="""
    You are a professional guide with a passion for travel.
    An organizational wizard who can turn a list of possibilities into a seamless itinerary.
    """,
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=llm,
    allow_delegation=False,
    )

In [6]:
from datetime import datetime

from_city = "India"
destination_city = "Rome"
date_from = "1st March 2025"
date_to = "7th March 2025"
interests = "sight seeing and good food"

In [9]:
from crewai import Task 

location_task = Task(
    description=f"""
    In French : This task involves a comprehensive data collection process to provide the traveler with essential information about their destination. It includes researching and compiling details on various accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa requirements, and any travel advisories that may be relevant.
    consider also the weather conditions forcast on the travel dates. and all the events that may be relevant to the traveler during the trip period.
    
    Traveling from : {from_city}
    Destination city : {destination_city}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1. if the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output=f"""
    if the {destination_city} is in a French country : Respond in FRENCH.
    In markdown format : A detailed markdown report that includes a curated list of recommended places to stay, a breakdown of daily living expenses, and practical travel tips to ensure a smooth journey.
    """,
    agent=location_expert,
    output_file='city_report.md',
)

guide_task = Task(
    description=f"""
    if the {destination_city} is in a French country : Respond in FRENCH.
    Tailored to the traveler's personal {interests}, this task focuses on creating an engaging and informative guide to the city's attractions. It involves identifying cultural landmarks, historical spots, entertainment venues, dining experiences, and outdoor activities that align with the user's preferences such {interests}. The guide also highlights seasonal events and festivals that might be of interest during the traveler's visit.
    Destination city : {destination_city}
    interests : {interests}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1. if the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output=f"""
    An interactive markdown report that presents a personalized itinerary of activities and attractions, complete with descriptions, locations, and any necessary reservations or tickets.
    """,

    agent=guide_expert,
    output_file='guide_report.md',
)

planner_task = Task(
    description=f"""
    This task synthesizes all collected information into a detaileds introduction to the city (description of city and presentation, in 3 pragraphes) cohesive and practical travel plan. and takes into account the traveler's schedule, preferences, and budget to draft a day-by-day itinerary. The planner also provides insights into the city's layout and transportation system to facilitate easy navigation.
    Destination city : {destination_city}
    interests : {interests}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1. if the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output="""
    if the {destination_city} is in a French country : Respond in FRENCH.
    A rich markdown document with emojis on each title and subtitle, that :
    In markdown format : 
    # Welcome to {destination_city} :
    A 4 paragraphes markdown formated including :
    - a curated articles of presentation of the city, 
    - a breakdown of daily living expenses, and spots to visit.
    # Here's your Travel Plan to {destination_city} :
    Outlines a daily detailed travel plan list with time allocations and details for each activity, along with an overview of the city's highlights based on the guide's recommendations
    """,
    context=[location_task, guide_task],
    #context=context,
    agent=planner_expert,
        output_file='travel_plan.md',
        )

def location_task(agent, from_city, destination_city, date_from, date_to):
    return Task(
        description=f"""
        In French : This task involves a comprehensive data collection process to provide the traveler with essential information about their destination. It includes researching and compiling details on various accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa requirements, and any travel advisories that may be relevant.
        consider also the weather conditions forcast on the travel dates. and all the events that may be relevant to the traveler during the trip period.
        
        Traveling from : {from_city}
        Destination city : {destination_city}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. if the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output=f"""
        if the {destination_city} is in a French country : Respond in FRENCH.
        In markdown format : A detailed markdown report that includes a curated list of recommended places to stay, a breakdown of daily living expenses, and practical travel tips to ensure a smooth journey.
        """,
        agent=agent,
        output_file='city_report.md',
    )

location_task = location_task(
  location_expert,
  from_city,
  destination_city,
  date_from,
  date_to
)

# Task: Location
def guide_task(agent, destination_city, interests, date_from, date_to):    
    return Task(
        description=f"""
        if the {destination_city} is in a French country : Respond in FRENCH.
        Tailored to the traveler's personal {interests}, this task focuses on creating an engaging and informative guide to the city's attractions. It involves identifying cultural landmarks, historical spots, entertainment venues, dining experiences, and outdoor activities that align with the user's preferences such {interests}. The guide also highlights seasonal events and festivals that might be of interest during the traveler's visit.
        Destination city : {destination_city}
        interests : {interests}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. if the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output=f"""
        An interactive markdown report that presents a personalized itinerary of activities and attractions, complete with descriptions, locations, and any necessary reservations or tickets.
        """,

        agent=agent,
        output_file='guide_report.md',
    )

guide_task = guide_task(
  guide_expert,
  destination_city,
  interests,
  date_from,
  date_to
)

# Task: Planner
def planner_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=f"""
        This task synthesizes all collected information into a detaileds introduction to the city (description of city and presentation, in 3 pragraphes) cohesive and practical travel plan. and takes into account the traveler's schedule, preferences, and budget to draft a day-by-day itinerary. The planner also provides insights into the city's layout and transportation system to facilitate easy navigation.
        Destination city : {destination_city}
        interests : {interests}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. if the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output="""
        if the {destination_city} is in a French country : Respond in FRENCH.
        A rich markdown document with emojis on each title and subtitle, that :
        In markdown format : 
        # Welcome to {destination_city} :
        A 4 paragraphes markdown formated including :
        - a curated articles of presentation of the city, 
        - a breakdown of daily living expenses, and spots to visit.
        # Here's your Travel Plan to {destination_city} :
        Outlines a daily detailed travel plan list with time allocations and details for each activity, along with an overview of the city's highlights based on the guide's recommendations
        """,
        #context=[location_task, guide_task],
        context=context,
        agent=agent,
        output_file='travel_plan.md',
        )

planner_task = planner_task(
  [location_task, guide_task],
  planner_expert,
  destination_city,
  interests,
  date_from,
  date_to,
)

In [13]:
from crewai import Crew, Process

crew = Crew(
    agents=[location_expert, guide_expert, planner_expert],
    tasks=[location_task, guide_task, planner_task],
    process=Process.sequential,
    full_output = True,
    share_crew=False,
    verbose=True
)

result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a54d9696-5d11-4717-b24f-0b73a5d908bc                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Trip Expert                                                                                      │
│                                                                                                                 │
│  Task:                                                                                                          │
│          In French : This task involves a comprehensive data collection process to provide the traveler with    │
│  essential information about their destination. It includes researching and compiling details on various        │
│  accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of       │
│  living in the area. The task also covers transportation options, visa requirements, and any travel advisories  │
│  that may be relevant.                                                                                          │
│          consider also the weather conditions forcast on the travel dates. and all the events that may be       │
│  relevant to the traveler during the trip period.                                                               │
│                                                                                                                 │
│          Traveling from : India                                                                                 │
│          Destination city : Rome                                                                                │
│          Arrival Date : 1st March 2025                                                                          │
│          Departure Date : 7th March 2025                                                                        │
│                                                                                                                 │
│          Follow this rules :                                                                                    │
│          1. if the Rome is in a French country : Respond in FRENCH.                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Sneh[WorkDirectory]\.venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Sneh[WorkDirectory]\.venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

[32;1m[1;3msnippet: May 23, 2025 · Review the latest State Department Travel Advisory about Italy. You can plan for
safe travel by following the entry and exit requirements, reviewing local laws, and other travel guidance from the 
U.S. embassy or consulate., title: Italy Travel Advisory | Travel.State.gov, link: 
https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories/italy-travel-advisory.html, snippet: 
Check the page for the country you plan to visit for any special advice notices and an up-date on the overall 
situation and any security or weather-related problems, epidemics, etc., title: Travel advice - Ministero degli 
Affari Esteri e della ..., link: https://www.esteri.it/en/sportello_info/domandefrequenti/viaggiare/, snippet: This
page is to assist travelers who have planned leisure or tourist trips to Italy. Different rules and regulations may
apply if your travel relates to other purposes such as business, family reasons etc. While every effort is made to 
keep this page up to date, please check advice from your airline and local authorities for the latest information 
... See full list on untolditaly.com To enter Italy, visitors must: 1. Have a minimum of 3 months validity on their
passport 2. Have a current and valid visa if required (no visa or visa waiver is currently required for citizens of
the United States, United Kingdom, Australia, or European Union) 3. There is no requirementto show proof of 
covid-19 vaccination See full list on untolditaly.com Is Italy allowing international visitors to enter the 
country? Yes, Italy welcomes visitors in 2024 provided you meet all entry requirements for your nationality and/or 
country of residence. To check what the situation is for your country of residence and nationality, please go to 
this Italian government website that aims to clarify the situation for you. You can also find a good summary of 
information for tourists at the Italian Tourist Board. What is the latest guidance on travel to Italy from my 
country? Currently the following advisories are in place. While every effort is made to keep this information 
current it is best to check with your country travel advisory in the “read more” links below. 1. United States – 
Level 2 – Exercise increased caution – read more 2. United Kingdom – Exercise caution – travel info. 3. Australia –
The Department of Foreign Affairs advises Australians to exercise a normal degree of caution in Italy – read more 
4. Canada – Take normal precautions in Italy – read m... Should I buy travel insurance to cover my trip? Travel 
insuranceis strongly recommended for all international travel however many travel insurance companies will not 
cover your trip for cancelations or health problems related to known health conditions. Check all policy inclusions
carefully before purchasing insurance. There are many reasons to take out travel insurance for Italy – you can read
about them in this article. No matter what decision you make, ensure you understand the risks associated with not 
having insurance and read the ter... See full list on untolditaly.com See full list on untolditaly.com Aug 22, 2025
· For Indian citizens looking to travel to Italy, securing a visa prior to arrival is mandatory. Please consult the
nearest Italy embassy or consulate for detailed visa application procedures. As of the latest update, travel 
restrictions for Rome Italy remain in place due to the ongoing COVID-19 pandemic. However, there are some essential
tips that can help travelers navigate these restrictions more effectively. Do Indian citizens need a visa for 
Italy? For Indian citizens looking to travel to Italy, securing a visa prior to arrival is mandatory . Please 
consult the nearest Italy embassy or consulate for detailed visa application procedures. Need help with your visa 
for Italy? Get your visa application processed quickly and easily through our trusted partner iVisa. What to do if 
a country has no Italian diplomatic m

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Planning Expert                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Welcome to Rome 🇮🇹:                                                                                          │
│                                                                                                                 │
│  Rome, the Eternal City, is a captivating blend of ancient history, stunning art, and vibrant modern life.      │
│  From the Colosseum and Roman Forum to the Vatican City and Trevi Fountain, Rome is overflowing with iconic     │
│  landmarks that whisper tales of emperors, artists, and popes. Beyond the historical sites, Rome boasts a       │
│  thriving culinary scene, offering delectable pasta dishes, crispy pizzas, and creamy gelato. The city's        │
│  charming neighborhoods, like Trastevere and Monti, invite you to wander through cobblestone streets, discover  │
│  hidden gems, and soak in the authentic Roman atmosphere.                                                       │
│                                                                                                                 │
│  A trip to Rome promises an unforgettable experience, immersing you in a world where the past and present       │
│  intertwine seamlessly. Imagine strolling through ancient ruins in the morning, indulging in a delicious plate  │
│  of pasta for lunch, and admiring Renaissance masterpieces in the afternoon. Rome is a city that awakens the    │
│  senses and leaves a lasting impression on every traveler.                                                      │
│                                                                                                                 │
│  Daily living expenses in Rome can vary depending on your travel style and preferences. On average, you can     │
│  expect to spend around €100-€200 per day. This includes accommodation (€50-€100 per night for a mid-range      │
│  hotel), food (€30-€50 per day), transportation (€10-€20 per day), and entrance fees to attractions (€20-€30    │
│  per day). To save money, consider staying in budget-friendly accommodations, eating at local trattorias, and   │
│  utilizing public transportation.                                                                               │
│                                                                                                                 │
│  Must-visit spots in Rome include the Colosseum, Roman Forum, Vatican City (St. Peter's Basilica and Vatican    │
│  Museums), Trevi Fountain, Pantheon, Spanish Steps, Borghese Gallery and Gardens, and the charming              │
│  neighborhoods of Trastevere and Monti. Don't forget to indulge in the local cuisine, such as pasta cacio e     │
│  pepe, carbonara, amatriciana, and Roman-style pizza. And be sure to try the delicious gelato – you'll find     │
│  excellent gelaterias throughout the city!                                                                      │
│                                                                                                                 │
│  # Here's your Travel Plan to Rome 🇮🇹:                                                                          │
│                                                                                                                 │
│  **Day 1: Arrival & Ancient Rome**                     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b380ee54-59b8-4c14-a9d5-0e8ad5fb72e7                                                                     │
│  Agent: Travel Planning Expert                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a54d9696-5d11-4717-b24f-0b73a5d908bc                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Welcome to Rome 🇮🇹:                                                                                          │
│                                                                                                                 │
│  Rome, the Eternal City, is a captivating blend of ancient history, stunning art, and vibrant modern life.      │
│  From the Colosseum and Roman Forum to the Vatican City and Trevi Fountain, Rome is overflowing with iconic     │
│  landmarks that whisper tales of emperors, artists, and popes. Beyond the historical sites, Rome boasts a       │
│  thriving culinary scene, offering delectable pasta dishes, crispy pizzas, and creamy gelato. The city's        │
│  charming neighborhoods, like Trastevere and Monti, invite you to wander through cobblestone streets, discover  │
│  hidden gems, and soak in the authentic Roman atmosphere.                                                       │
│                                                                                                                 │
│  A trip to Rome promises an unforgettable experience, immersing you in a world where the past and present       │
│  intertwine seamlessly. Imagine strolling through ancient ruins in the morning, indulging in a delicious plate  │
│  of pasta for lunch, and admiring Renaissance masterpieces in the afternoon. Rome is a city that awakens the    │
│  senses and leaves a lasting impression on every traveler.                                                      │
│                                                                                                                 │
│  Daily living expenses in Rome can vary depending on your travel style and preferences. On average, you can     │
│  expect to spend around €100-€200 per day. This includes accommodation (€50-€100 per night for a mid-range      │
│  hotel), food (€30-€50 per day), transportation (€10-€20 per day), and entrance fees to attractions (€20-€30    │
│  per day). To save money, consider staying in budget-friendly accommodations, eating at local trattorias, and   │
│  utilizing public transportation.                                                                               │
│                                                                                                                 │
│  Must-visit spots in Rome include the Colosseum, Roman Forum, Vatican City (St. Peter's Basilica and Vatican    │
│  Museums), Trevi Fountain, Pantheon, Spanish Steps, Borghese Gallery and Gardens, and the charming              │
│  neighborhoods of Trastevere and Monti. Don't forget to indulge in the local cuisine, such as pasta cacio e     │
│  pepe, carbonara, amatriciana, and Roman-style pizza. And be sure to try the delicious gelato – you'll find     │
│  excellent gelaterias throughout the city!                                                                      │
│                                                                                                                 │
│  # Here's your Travel Plan to Rome 🇮🇹:                                                                          │
│                                                       

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 